In [1]:
import magpylib as magpy
import sqlite3
import pandas as pd
con = sqlite3.connect('database.db')
cur = con.cursor()

In [6]:
df = pd.read_sql_query("SELECT * from mag_positions", con)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df[df['Timestamp'] - pd.Timestamp.now() < pd.Timedelta(5, "hours")]
df.drop_duplicates(subset = ['pos'], inplace=True)
df

,pos,m,Timestamp
0,"83, -60, 47","-930788.0, 0.0, 0.0",2022-04-25 00:00:30
2,"83, -60, 43","930788.0, 0.0, 0.0",2022-04-25 00:00:36
4,"78, -60, 43","0.0, 0.0, -930788.0",2022-04-25 00:00:40
6,"76, -60, 39","-930788.0, 0.0, 0.0",2022-04-25 00:00:43
8,"76, -60, 37","-930788.0, 0.0, 0.0",2022-04-25 00:02:14


In [12]:
output_df = pd.read_sql_query("SELECT * from mag_field", con)
output_df

,pos,mag_field


In [153]:
pos = df['pos'].str.split(",", expand=True).astype(float)
m = df['m'].str.split(",", expand=True).astype(float)

In [154]:
pos

,0,1,2
0,79.0,-60.0,73.0
2,79.0,-60.0,75.0
4,79.0,-60.0,77.0
6,82.0,-60.0,75.0
8,82.0,-60.0,73.0
10,79.0,-59.0,75.0
11,93.0,-60.0,74.0
13,97.0,-60.0,66.0
15,95.0,-60.0,56.0
17,87.0,-60.0,54.0


In [86]:
def check_if_already_calculated():
    
    
    
    return if_calculated

In [203]:
def look_at_neighbors(x, y, z,bounds):
    moments = []
    neighbors = pos[(pos[0] <= x + bounds) & (pos[0] >= x - bounds) &
                     (pos[1] <= y + bounds) & (pos[1] >= y - bounds) &
                     (pos[2] <= z + bounds) & (pos[2] >= z - bounds)]
    
    for i in neighbors.index:
        moments.append(m.loc[i])
    return moments, neighbors

def a_block_away(x, y, z, bounds):    
    moments = []
    centers = []
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                block = pos[(pos[0] <= x + bounds + 4 * i) & (pos[0] >= x - bounds + 4 * i) &
                            (pos[1] <= y + bounds + 4 * j) & (pos[1] >= y - bounds + 4 * j) &
                            (pos[2] <= z + bounds + 4 * k) & (pos[2] >= z - bounds + 4 * k)]
                moment = m.loc[block.index].sum()
                moments.append(moment)
         
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                centers.append([x + 4*i, y + 4*j, z + 4*k])
    
    return moments, centers

def prepare_df(x,y,z,bounds):
    
    moments, centers = a_block_away(x,y,z,bounds)
    neighbor_moments, neighbors = look_at_neighbors(x,y,z,bounds)
    centers = pd.concat([pd.DataFrame(centers), pd.DataFrame(neighbors)], ignore_index=True)
    moments = pd.concat([pd.DataFrame(moments), pd.DataFrame(neighbor_moments)], ignore_index=True)
    
    return centers, moments

def get_coords(pos):
    coords = pd.DataFrame({"x":[], "y":[], "z":[]})
    for index in pos.index:
        for i in range(-3,4):
            for j in range(-3,4):
                for k in range(-3,4):
                    x = pos[0].loc[index] + i
                    y = pos[1].loc[index] + j
                    z = pos[2].loc[index] + k
                    if coords[(coords["x"] == i) & (coords["y"] == j) & (coords["z"] == k)].empty:
                        coords_ = pd.DataFrame({"x":[x], "y":[y], "z":[z]})
                        coords = pd.concat([coords, coords_], ignore_index=True)
    return coords

coords = get_coords(pos)
mag_fields = []
for i in coords.index:
    x = coords["x"].loc[i]
    y = coords["y"].loc[i]
    z = coords["z"].loc[i]
    centers, moments = prepare_df(x,y,z,1)
    magnets = []
    for i in centers.index:
        magnets.append(magpy.magnet.Cuboid(magnetization=(moments[0].loc[i], moments[1].loc[i], moments[2].loc[i]),
                                           dimension = (1000, 1000, 1000),
                                           position=(centers[0].loc[i]*1000, centers[1].loc[i]*1000, centers[2].loc[i]*1000)))
    sensor = magpy.Sensor(position=(x*1000, y*1000, z*1000))
    B = sensor.getB(magnets, sumup = True)
    mag_fields.append(B)

print(mag_fields)

[array([-1.23350922e-11, -2.22744421e+02, -2.22744421e+02]), array([-1.23350922e-11, -2.22744421e+02, -2.22744421e+02]), array([0., 0., 0.]), array([-204.5792609 , -836.57010929, -222.74442126]), array([-204.5792609 , -836.57010929, -222.74442126]), array([-204.5792609 , -836.57010929, -222.74442126]), array([0., 0., 0.]), array([-204.5792609 , -836.57010929,  222.74442126]), array([-204.5792609 , -836.57010929,  222.74442126]), array([-1.23350922e-11, -2.22744421e+02, -2.22744421e+02]), array([-1.23350922e-11, -2.22744421e+02, -2.22744421e+02]), array([0., 0., 0.]), array([-204.5792609 , -836.57010929, -222.74442126]), array([-204.5792609 , -836.57010929, -222.74442126]), array([-204.5792609 , -836.57010929, -222.74442126]), array([0., 0., 0.]), array([-204.5792609 , -836.57010929,  222.74442126]), array([-204.5792609 , -836.57010929,  222.74442126]), array([0., 0., 0.]), array([1.23350922e-11, 2.22744421e+02, 2.22744421e+02]), array([1.23350922e-11, 2.22744421e+02, 2.22744421e+02]), 

In [215]:
coords_df = pd.DataFrame(coords)
mag_field_df = pd.DataFrame(mag_fields)
output_df = coords_df.join(mag_field_df)
output_df[0]

0      -1.233509e-11
1      -1.233509e-11
2       0.000000e+00
3      -2.045793e+02
4      -2.045793e+02
            ...     
8014    2.045793e+02
8015    4.091585e+02
8016    2.045793e+02
8017    2.045793e+02
8018    1.233509e-11
Name: 0, Length: 8019, dtype: float64

In [220]:
output_df['pos'] = output_df['x'].astype(str) + "," + output_df['y'].astype(str) + "," + output_df['z'].astype(str)
output_df['mag_field'] = output_df[0].astype(str) + "," + output_df[1].astype(str) + "," + output_df[2].astype(str)

output_df[['pos','mag_field']].to_sql("mag_field", con = con, if_exists='append', index = False)

8019

In [198]:
if pos[pos[0] == 0].empty:
    print('DataFrame is empty!')

DataFrame is empty!


In [202]:
centers

,0,1,2
0,71.0,-68.0,65.0
1,71.0,-68.0,69.0
2,71.0,-68.0,73.0
3,71.0,-64.0,65.0
4,71.0,-64.0,69.0
5,71.0,-64.0,73.0
6,71.0,-60.0,65.0
7,71.0,-60.0,69.0
8,71.0,-60.0,73.0
9,75.0,-68.0,65.0


In [7]:
bounds = 3
cube_list = []
coords = pos.iloc[0]
mag_blocks = pos[(pos[0] <= pos[0].iloc[0] + bounds) & (pos[0] >= pos[0].iloc[0] - bounds) &
                 (pos[1] <= pos[1].iloc[0] + bounds) & (pos[1] >= pos[1].iloc[0] - bounds) &
                 (pos[2] <= pos[2].iloc[0] + bounds) & (pos[2] >= pos[2].iloc[0] - bounds)]


            

0    79.0
1   -60.0
2    73.0
Name: 0, dtype: float64


In [13]:
bounds = 6
coords = pos.iloc[0]
pos[(pos[0] <= pos[0].iloc[0] + bounds) & (pos[0] >= pos[0].iloc[0] - bounds) &
   (pos[1] <= pos[1].iloc[0] + bounds) & (pos[1] >= pos[1].iloc[0] - bounds) &
   (pos[2] <= pos[2].iloc[0] + bounds) & (pos[2] >= pos[2].iloc[0] - bounds)]

,0,1,2
0,79.0,-60.0,73.0
2,79.0,-60.0,75.0
4,79.0,-60.0,77.0
5,79.0,-60.0,77.0
6,82.0,-60.0,75.0
7,82.0,-60.0,75.0
8,82.0,-60.0,73.0
9,82.0,-60.0,73.0
10,79.0,-59.0,75.0


In [35]:
m

,0,1,2
0,-930788.0,0.0,0.0
2,-930788.0,0.0,0.0
4,-930788.0,0.0,0.0
6,-930788.0,0.0,0.0
8,-930788.0,0.0,0.0
10,-930788.0,0.0,0.0
12,-930788.0,0.0,0.0
14,-930788.0,0.0,0.0
16,-930788.0,0.0,0.0
18,-930788.0,0.0,0.0


In [28]:
for i in range(len(pos)):
    print(m[0].iloc[i])

-930788.0
-930788.0
-930788.0
-930788.0
-930788.0
-930788.0
-930788.0
-930788.0
-930788.0
-930788.0
0.0
0.0
0.0
0.0
0.0
0.0


In [41]:
list = []
for i in range(len(pos)):
    list.append(magpy.magnet.Cuboid(magnetization=(m[0].iloc[i], m[1].iloc[i], m[2].iloc[i]), dimension = (1000, 1000, 1000), position=(pos[0].iloc[i], pos[1].iloc[i], pos[2].iloc[i])))
sensor = magpy.Sensor(position=(0,0,0))


B = sensor.getB(list, sumup = True)
print(type(cube1))
print(B)

<class 'magpylib._src.obj_classes.class_mag_Cuboid.Cuboid'>
[ 0.52361492  0.39424903 -0.84690328]
